<a href="https://colab.research.google.com/github/jay2jaykp/ML/blob/master/LogisticRegressionTimestemp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from tqdm import tqdm
#from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import partial
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import time
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from collections import deque

In [58]:
df = pd.read_csv('smalltrace.csv', sep='\t')
df.columns = ['no','timestamp','pid','pname','bno', 'bsize', 'op', 'dvmajor', 'dvminor', 'blockhash']
df.head()

,no,timestamp,pid,pname,bno,bsize,op,dvmajor,dvminor,blockhash
0,0,89966811921625,4257,nfsd,488846200,8,W,6,0,2c4787c7671c2f638777c4ab72859ce1
1,1,89967003980393,4253,nfsd,508455704,8,W,6,0,88b93b628d84082186026d9da044f173
2,2,89967004049935,4253,nfsd,508455712,8,W,6,0,b5e9f4e5ab62a4fff5313a606b0ad4e3
3,3,89967004093615,4253,nfsd,508455720,8,W,6,0,7af5556ad4d45263bcd8358867333dd4
4,4,89967324150054,4254,nfsd,508500288,8,W,6,0,88b93b628d84082186026d9da044f173


In [39]:
df.at[0,'timestamp']

89966811921625

In [0]:
blocktrace = df['bno'].tolist()
seq = df['no'].tolist()
#convert element to int from str
sequences = np.array([int(x) for x in seq])
blocktrace = np.array([int(x) for x in blocktrace])

In [0]:
''' 
  Input: Cache sequence [1, 2, 3]. get cache blocks by df['bno'][cache sequence #]
  get timestamp
  df['timestamp'][1], df['timestamp'][2], df['timestamp'][3]
  find avg. (avgTS)
  get relative => (avgTS - df['timestamp'][A])
  Output: [relativeA, relativeB, relativeC]
'''

def getRelativeTS(Cache_indices):
  tfsm=np.zeros((len(Cache_indices)))
  for i in Cache_indices:
    tfsm [Cache_indices.index(i)] = df.timestamp[i]
  
  avgTS = np.mean(tfsm)
  relTS = avgTS-tfsm
  
  return list(relTS)    
  #df.at[]

In [25]:
l=[0,1]
df.at[1,1]
#t=np.zeros((4,1))
t=np.array([1,2,3])
b=4
t=list(np.append(t,b))
t
np.mean([1,2,3])

2.0

In [0]:
  
''' 
  Input: Input: Cache sequence [1, 2, 3]. get cache blocks [A,B,C] by df['bno'][cache sequence #]
  gdaAbno
  df['bno'][1], df['bno'][2], df['bno'][3]
''''
def getRelativePos():

In [0]:


  

'''
get the furthest accessed block. Scans OPT dictionary and selects the maximum positioned element
'''

def getFurthestAccessBlock(C, OPT):
    maxAccessPosition = -1
    maxAccessBlock = -1
    for cached_block in C:
        if len(OPT[cached_block]) is 0:
            #print ( "Not Acccessing block anymore " + str(cached_block))
            return cached_block            
    for cached_block in C:  
        if OPT[cached_block][0] > maxAccessPosition:
            maxAccessPosition = OPT[cached_block][0]
            maxAccessBlock = cached_block
    #print ( "chose to evict " + str(maxAccessBlock) + " since its position of access is " + str(maxAccessPosition))
    return maxAccessBlock

def belady_opt(blocktrace,sequences,frame):
    OPT = defaultdict(partial(np.ndarray,0))

    for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        OPT[block] = np.append(OPT[block], i)    

    #print ("created OPT dictionary")    

    hit, miss = 0, 0

    C = deque()
    C2 = deque()
    d = defaultdict(deque)
    d_ftime = set()
    d_timestamp = {}
    d_label = {}
    
    for k,block in enumerate(tqdm(blocktrace, desc="OPT", leave=False)):

        if block in C:
            #OPT[block] = OPT[block][1:]
            OPT[block] = np.delete(OPT[block],0)
            hit+=1
            #print('hit' + str(block))
            #print(OPT)
        else:
            #print('miss' + str(block))
            miss+=1
            if len(C) == frame:
                fblock = getFurthestAccessBlock(C, OPT)
                assert(fblock != -1)
                d[fblock] = deque(zip(C,C2)) 
                d_timestamp[fblock] = getRelativeTS(list(C2)) #Returning Time Stemp
                d_label[fblock] = C.index(fblock)
                C2.remove(C2[C.index(fblock)])
                C.remove(fblock)
                #C2.remove()
                d_ftime.add(sequences[k])
            C.append(block)
            C2.append(k)
            d_ftime.add(sequences[k])
            #OPT[block] = OPT[block][1:]
            #print(OPT)
            OPT[block] = np.delete(OPT[block],[0])

    #print ("hit count" + str(hit_count))
    #print ("miss count" + str(miss_count))
    hitrate = hit / (hit + miss)
    print(hitrate)
    return d,d_timestamp,d_label

In [104]:
d,d_timestamp,d_label = belady_opt(blocktrace,sequences, 10)

OPT: building index: 100%|██████████| 10000/10000 [00:00<00:00, 136479.13it/s]
                                                           

0.0763


In [90]:
d[14254312]

deque([(485954192, 7317),
       (486242128, 7323),
       (486242136, 7324),
       (485954216, 7329),
       (485954224, 7330),
       (485954232, 7331),
       (485954200, 7359),
       (485954208, 7360),
       (14254312, 7364),
       (283460608, 7638)])

In [105]:
d_label[14254312]

8

In [106]:
 len(d_timestamp)

9197

In [125]:
#Time Stamp only (Training data)
data = []
for i,j in d_timestamp.items():
  dat = list(map(str, j))
  data.append(' '.join(dat))
  #label.append(i)
len(data)

9197

In [130]:
#Time Stamp only (Label data)
label = []
for i,j in d_label.items():
  label.append(j)
len(label)

9197

In [131]:
#CountVEctorizer for fitting the data
vec = CountVectorizer(binary=True)
vec_ = vec.fit(data)
DATA = vec_.transform(data)
DATA

<9197x91311 sparse matrix of type '<class 'numpy.int64'>'
	with 106588 stored elements in Compressed Sparse Row format>

In [0]:
#Train-Test split
X_train, X_test, Y_train, Y_test = train_test_split(DATA, label ,test_size=0.5, shuffle=True)

In [0]:
#Fitting Logistic Regression Model
clf = LogisticRegression().fit(X_train, Y_train)

In [139]:
#Accuracy Testing
accuracy_score(Y_test, clf.predict(X_test))

0.8269188954120461